# .NET Initialization
<div style="width:100%;height:6px;background-color:Black;"></div>

MiniSim is a .net dll and as such we need to import the pythonnet module (clr) and load the .net DLL into the Python kernel. Afterwards we can import the .net namespaces as Python modules.

In [1]:
import clr
clr.AddReference(r"..\bin\MiniSim.Core")

import MiniSim.Core.Expressions as expr
import MiniSim.Core.Flowsheeting as fl
import MiniSim.Core.Numerics as num
from MiniSim.Core.UnitsOfMeasure import Unit, SI, METRIC, PhysicalDimension
import MiniSim.Core.ModelLibrary as lib
import MiniSim.Core.PropertyDatabase as chemsep
from MiniSim.Core.Reporting import Generator, StringBuilderLogger
import MiniSim.Core.Thermodynamics as thermo

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams['grid.color'] = 'k'

# General Objects Instantiation
<div style="width:100%;height:6px;background-color:Black;"></div>

In [4]:
Database = chemsep.ChemSepAdapter()
logger = StringBuilderLogger();
reporter = Generator(logger)

Database.ListComponents("benzen")


['1,2,4-trichlorobenzene',
 'M-dichlorobenzene',
 'O-dichlorobenzene',
 'P-dichlorobenzene',
 'Bromobenzene',
 'Monochlorobenzene',
 'Iodobenzene',
 'Nitrobenzene',
 'Benzene',
 'Ethylbenzene',
 'N-propylbenzene',
 'N-butylbenzene',
 '1,2,3-trimethylbenzene',
 '1,2,4-trimethylbenzene',
 'Isobutylbenzene',
 'Sec-butylbenzene',
 'Tert-butylbenzene',
 'O-diethylbenzene',
 'M-diethylbenzene',
 'P-diethylbenzene',
 '1,2,3,4-tetramethylbenzene',
 '1,2,3,5-tetramethylbenzene',
 '1,2,4,5-tetramethylbenzene',
 '1-methyl-3-n-propylbenzene',
 '1-methyl-4-n-propylbenzene',
 'P-diisopropylbenzene']

# Definition of the Thermo-System
<div style="width:100%;height:6px;background-color:Black;"></div>

When an instance of the ThermodynamicSystem class is created, we can define the base method (NRTL. UNIQUAC or SRK). This will set the internal states accordingly. Alternatively the equilibrium approach could be chosen explicitly with the respective data classes. 

We then add three compontents from the ChemSep Database, Benzene, Toluene and P-Xylene. Use the ListComponents(string) method to find suitable components, and then use the exact spelling to select it from the databank.

In [5]:
sys= thermo.ThermodynamicSystem("Test2","NRTL", "default")
sys.AddComponent(Database.FindComponent("Benzene"))
sys.AddComponent(Database.FindComponent("Toluene"))
sys.AddComponent(Database.FindComponent("P-xylene"))
Database.FillBIPs(sys)

sys.VariableFactory.SetOutputDimension(PhysicalDimension.Pressure, METRIC.mbar)

# Flowsheet Definition
<div style="width:100%;height:6px;background-color:Black;"></div>

First we create the feed stream S01. For this stream, we specify the temperature and pressure, as well as the total molar flow and the molar composition. 

If ALL molar fractions are specified, the summation closure equation is not added to the equation system. In that case, the composition must add up to exactly one. In this example, we initialize the molar fraction of P-xylene, which means that it is still a free variable, and as such the closure will be added to the equation system.

We need to define our own unit of measure for the molar flow, but because Pythonnet does not support .net operator overlading, we have to use a helper function that creates an aggregate unit: kmol/h

In [6]:
kmolh=Unit.Make([SI.kmol],[SI.h])
print(kmolh)

kmol/h


In [7]:
S01 = (fl.MaterialStream("S01", sys)
    .Specify("T",50, METRIC.C)
    .Specify("P",1, METRIC.bar)
    .Specify("n",10.0, kmolh)
    .Specify("x[Benzene]",0.5)
    .Specify("x[Toluene]",0.1)
    .Init("x[P-xylene]",0.4)
    .InitializeFromMolarFractions()
    .FlashPT())

S02 = fl.MaterialStream("S02",sys)
S03 = fl.MaterialStream("S03",sys)

Once the stream objects are created, we can instantiate the Flash object. We need to connect the In, Vap and Liq port to their respective streams. For a Flash, we need two additional degrees of freedom, chosen from temperature T, pressure P, vapor fraction VF or heat duty Q.

In [8]:
flash = (lib.Flash("Flash",sys)
    .Connect("In", S01)
    .Connect("Vap", S02)
    .Connect("Liq", S03)
    .Specify("P", 1, METRIC.bar)
    .Specify("VF",0.5)
    .Initialize())

Now that all the objects are created we can assemble them in a flowsheet.

In [9]:
flowsheet= (fl.Flowsheet("Flow")
    .AddMaterialStreams(S01,S02,S03)
    .AddUnit(flash))

# Problem Solving using Newton-Raphson
<div style="width:100%;height:6px;background-color:Black;"></div>

First, we create an instance of the solver class, which requires an instance of any ILogger-implementing class. Then we call the Solve method and pass either a flowsheet or an equation system. The equation system is solved using the Newton-Raphson method.
Finally we use the Reporter object to write a textual summary of the flowsheet.

In [10]:
presolver=  num.BasicNewtonSolver(logger)
presolver.Solve(flowsheet)

reporter.Report(flowsheet, 4, False)
print(logger.Flush())

Iter Step Length     Infeasibility   Damping Notes
0    0               7816,8          1       
1    25253,7         2038,62         0,739200282809472 
2    5078,88         0,0920979       1       
3    678,222         0,00135012      1       
4    7,01157         2,38145E-05     1       
5    0,0425876       8,6519E-09      1       
Problem NLAES was solved. Constraint violation is below tolerance (5 iter, 53,42 ms, NV = 98, NZ = 347, NZ% = 3,61%)

Report for flowsheet Flow

Material Streams

System                    Test2      

Property                  Unit                S01          S02          S03

Temperature               °C              50.0000     106.4570     106.4570
Pressure                  mbar          1000.0000    1000.0000    1000.0000
Vapor Fraction            mol/mol          0.0000       1.0000       0.0000
Specific Enthalpy         kJ/kmol     -33624.2220    8757.1804  -24413.0090
Phase                                      Liquid        Vapor  LiquidVapor

Tot

# Equation Analyis

When we want to view the actual equations used in the model, we have to create an EquationSystem explicitly.

In [11]:
from IPython.display import display, Markdown, Latex
eq= num.AlgebraicSystem("Review")
flowsheet.CreateEquations(eq)
eqtab=""
eqtab+="|Model|Class|Group|Equation<img width=500/>|Residual|\n"
eqtab+="|:----|:----|:----|:------|---|\n"
for e in eq.Equations:
    eqtab+="|"+e.ModelName+"|"+e.ModelClass+"|"+e.Group+"|"+e.Pretty()+"|"+str(e.Residual())+"\n"
display(Markdown(eqtab))

|Model|Class|Group|Equation<img width=500/>|Residual|
|:----|:----|:----|:------|---|
|S01|MaterialStream|Closure for Bulk Composition|  $(n_{Benzene} + n_{Toluene} + n_{P-xylene}) - n = 0$|0.0
|S01|MaterialStream||  $n_{Benzene} - x_{Benzene} * n = 0$|0.0
|S01|MaterialStream||  $n_{Toluene} - x_{Toluene} * n = 0$|0.0
|S01|MaterialStream||  $n_{P-xylene} - x_{P-xylene} * n = 0$|0.0
|S01|MaterialStream||  $nL_{Benzene} - xL_{Benzene} * nL = 0$|0.0
|S01|MaterialStream||  $nL_{Toluene} - xL_{Toluene} * nL = 0$|0.0
|S01|MaterialStream||  $nL_{P-xylene} - xL_{P-xylene} * nL = 0$|0.0
|S01|MaterialStream||  $nV_{Benzene} - xV_{Benzene} * nV = 0$|0.0
|S01|MaterialStream||  $nV_{Toluene} - xV_{Toluene} * nV = 0$|0.0
|S01|MaterialStream||  $nV_{P-xylene} - xV_{P-xylene} * nV = 0$|0.0
|S01|MaterialStream|Mole Balance|  $VF * n - nV = 0$|0.0
|S01|MaterialStream|Total Mass Balance|  $n - nL - nV = 0$|0.0
|S01|MaterialStream|Enthalpy Balance|  $\frac{(VF * hV + (1 - VF) * hL)}{1000} - \frac{h}{1000} = 0$|0.0
|S01|MaterialStream|Equilibrium|  $FlashZ((xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - (xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}),(xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}) - 1,(xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - 1) = 0$|0.0
|S01|MaterialStream|Component-Balance|  $1 - VF * xL_{Benzene} + VF * xV_{Benzene} - x_{Benzene} = 0$|0.0
|S01|MaterialStream|Equilibrium|  $K_{Benzene} * xL_{Benzene} - xV_{Benzene} = 0$|0.0
|S01|MaterialStream|Component-Balance|  $1 - VF * xL_{Toluene} + VF * xV_{Toluene} - x_{Toluene} = 0$|0.0
|S01|MaterialStream|Equilibrium|  $K_{Toluene} * xL_{Toluene} - xV_{Toluene} = 0$|0.0
|S01|MaterialStream|Component-Balance|  $1 - VF * xL_{P-xylene} + VF * xV_{P-xylene} - x_{P-xylene} = 0$|0.0
|S01|MaterialStream|Equilibrium|  $K_{P-xylene} * xL_{P-xylene} - xV_{P-xylene} = 0$|0.0
|S02|MaterialStream|Closure for Bulk Composition|  $(n_{Benzene} + n_{Toluene} + n_{P-xylene}) - n = 0$|2.220446049250313e-16
|S02|MaterialStream||  $n_{Benzene} - x_{Benzene} * n = 0$|0.0
|S02|MaterialStream||  $n_{Toluene} - x_{Toluene} * n = 0$|0.0
|S02|MaterialStream||  $n_{P-xylene} - x_{P-xylene} * n = 0$|0.0
|S02|MaterialStream||  $nL_{Benzene} - xL_{Benzene} * nL = 0$|0.0
|S02|MaterialStream||  $nL_{Toluene} - xL_{Toluene} * nL = 0$|0.0
|S02|MaterialStream||  $nL_{P-xylene} - xL_{P-xylene} * nL = 0$|0.0
|S02|MaterialStream||  $nV_{Benzene} - xV_{Benzene} * nV = 0$|-1.1102230246251565e-16
|S02|MaterialStream||  $nV_{Toluene} - xV_{Toluene} * nV = 0$|0.0
|S02|MaterialStream||  $nV_{P-xylene} - xV_{P-xylene} * nV = 0$|-5.551115123125783e-17
|S02|MaterialStream|Mole Balance|  $VF * n - nV = 0$|-3.622686595150526e-10
|S02|MaterialStream|Total Mass Balance|  $n - nL - nV = 0$|-3.622686595150526e-10
|S02|MaterialStream|Enthalpy Balance|  $\frac{(VF * hV + (1 - VF) * hL)}{1000} - \frac{h}{1000} = 0$|-8.651898752987108e-09
|S02|MaterialStream|Equilibrium|  $FlashZ((xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - (xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}),(xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}) - 1,(xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - 1) = 0$|0.0
|S02|MaterialStream|Component-Balance|  $1 - VF * xL_{Benzene} + VF * xV_{Benzene} - x_{Benzene} = 0$|-9.250578081321237e-11
|S02|MaterialStream|Equilibrium|  $K_{Benzene} * xL_{Benzene} - xV_{Benzene} = 0$|-3.774758283725532e-15
|S02|MaterialStream|Component-Balance|  $1 - VF * xL_{Toluene} + VF * xV_{Toluene} - x_{Toluene} = 0$|3.2056302057270614e-12
|S02|MaterialStream|Equilibrium|  $K_{Toluene} * xL_{Toluene} - xV_{Toluene} = 0$|1.8318679906315083e-15
|S02|MaterialStream|Component-Balance|  $1 - VF * xL_{P-xylene} + VF * xV_{P-xylene} - x_{P-xylene} = 0$|8.930006734075846e-11
|S02|MaterialStream|Equilibrium|  $K_{P-xylene} * xL_{P-xylene} - xV_{P-xylene} = 0$|-3.885780586188048e-16
|S03|MaterialStream|Closure for Bulk Composition|  $(n_{Benzene} + n_{Toluene} + n_{P-xylene}) - n = 0$|0.0
|S03|MaterialStream||  $n_{Benzene} - x_{Benzene} * n = 0$|-5.551115123125783e-17
|S03|MaterialStream||  $n_{Toluene} - x_{Toluene} * n = 0$|2.7755575615628914e-17
|S03|MaterialStream||  $n_{P-xylene} - x_{P-xylene} * n = 0$|0.0
|S03|MaterialStream||  $nL_{Benzene} - xL_{Benzene} * nL = 0$|-5.551115123125783e-17
|S03|MaterialStream||  $nL_{Toluene} - xL_{Toluene} * nL = 0$|0.0
|S03|MaterialStream||  $nL_{P-xylene} - xL_{P-xylene} * nL = 0$|-1.1102230246251565e-16
|S03|MaterialStream||  $nV_{Benzene} - xV_{Benzene} * nV = 0$|9.75722606397663e-25
|S03|MaterialStream||  $nV_{Toluene} - xV_{Toluene} * nV = 0$|3.0118762247374504e-25
|S03|MaterialStream||  $nV_{P-xylene} - xV_{P-xylene} * nV = 0$|-1.2769102254047342e-24
|S03|MaterialStream|Mole Balance|  $VF * n - nV = 0$|1.3240816808652208e-23
|S03|MaterialStream|Total Mass Balance|  $n - nL - nV = 0$|-3.4754052987492554e-17
|S03|MaterialStream|Enthalpy Balance|  $\frac{(VF * hV + (1 - VF) * hL)}{1000} - \frac{h}{1000} = 0$|6.623253057114198e-10
|S03|MaterialStream|Equilibrium|  $FlashZ((xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - (xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}),(xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}) - 1,(xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - 1) = 0$|0.0
|S03|MaterialStream|Component-Balance|  $1 - VF * xL_{Benzene} + VF * xV_{Benzene} - x_{Benzene} = 0$|0.0
|S03|MaterialStream|Equilibrium|  $K_{Benzene} * xL_{Benzene} - xV_{Benzene} = 0$|-1.3322676295501878e-15
|S03|MaterialStream|Component-Balance|  $1 - VF * xL_{Toluene} + VF * xV_{Toluene} - x_{Toluene} = 0$|0.0
|S03|MaterialStream|Equilibrium|  $K_{Toluene} * xL_{Toluene} - xV_{Toluene} = 0$|-2.636779683484747e-16
|S03|MaterialStream|Component-Balance|  $1 - VF * xL_{P-xylene} + VF * xV_{P-xylene} - x_{P-xylene} = 0$|0.0
|S03|MaterialStream|Equilibrium|  $K_{P-xylene} * xL_{P-xylene} - xV_{P-xylene} = 0$|9.71445146547012e-16
|VLEQ|MaterialStream|Closure for Bulk Composition|  $(n_{Benzene} + n_{Toluene} + n_{P-xylene}) - n = 0$|0.0
|VLEQ|MaterialStream||  $n_{Benzene} - x_{Benzene} * n = 0$|0.0
|VLEQ|MaterialStream||  $n_{Toluene} - x_{Toluene} * n = 0$|0.0
|VLEQ|MaterialStream||  $n_{P-xylene} - x_{P-xylene} * n = 0$|0.0
|VLEQ|MaterialStream||  $nL_{Benzene} - xL_{Benzene} * nL = 0$|-5.551115123125783e-17
|VLEQ|MaterialStream||  $nL_{Toluene} - xL_{Toluene} * nL = 0$|2.7755575615628914e-17
|VLEQ|MaterialStream||  $nL_{P-xylene} - xL_{P-xylene} * nL = 0$|0.0
|VLEQ|MaterialStream||  $nV_{Benzene} - xV_{Benzene} * nV = 0$|0.0
|VLEQ|MaterialStream||  $nV_{Toluene} - xV_{Toluene} * nV = 0$|-2.7755575615628914e-17
|VLEQ|MaterialStream||  $nV_{P-xylene} - xV_{P-xylene} * nV = 0$|5.551115123125783e-17
|VLEQ|MaterialStream|Mole Balance|  $VF * n - nV = 0$|0.0
|VLEQ|MaterialStream|Total Mass Balance|  $n - nL - nV = 0$|0.0
|VLEQ|MaterialStream|Enthalpy Balance|  $\frac{(VF * hV + (1 - VF) * hL)}{1000} - \frac{h}{1000} = 0$|3.311591001420311e-10
|VLEQ|MaterialStream|Equilibrium|  $FlashZ((xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - (xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}),(xV_{Benzene} + xV_{Toluene} + xV_{P-xylene}) - 1,(xL_{Benzene} + xL_{Toluene} + xL_{P-xylene}) - 1) = 0$|-2.220446049250313e-16
|VLEQ|MaterialStream|Component-Balance|  $1 - VF * xL_{Benzene} + VF * xV_{Benzene} - x_{Benzene} = 0$|0.0
|VLEQ|MaterialStream|Equilibrium|  $K_{Benzene} * xL_{Benzene} - xV_{Benzene} = 0$|-1.3322676295501878e-15
|VLEQ|MaterialStream|Component-Balance|  $1 - VF * xL_{Toluene} + VF * xV_{Toluene} - x_{Toluene} = 0$|0.0
|VLEQ|MaterialStream|Equilibrium|  $K_{Toluene} * xL_{Toluene} - xV_{Toluene} = 0$|-2.7755575615628914e-16
|VLEQ|MaterialStream|Component-Balance|  $1 - VF * xL_{P-xylene} + VF * xV_{P-xylene} - x_{P-xylene} = 0$|0.0
|VLEQ|MaterialStream|Equilibrium|  $K_{P-xylene} * xL_{P-xylene} - xV_{P-xylene} = 0$|9.992007221626409e-16
|Flash|Flash|Mass Balance|  $n_{Benzene} - n_{Benzene} + n_{Benzene} = 0$|0.0
|Flash|Flash|Mass Balance|  $n_{Toluene} - n_{Toluene} + n_{Toluene} = 0$|0.0
|Flash|Flash|Mass Balance|  $n_{P-xylene} - n_{P-xylene} + n_{P-xylene} = 0$|0.0
|Flash|Flash|Pressure drop|  $\frac{P}{10000} - \frac{(P - DP)}{10000} = 0$|0.0
|Flash|Flash|Pressure Balance|  $\frac{P}{10000} - \frac{(P)}{10000} = 0$|0.0
|Flash|Flash|Temperature Balance|  $\frac{T}{1000} - \frac{T}{1000} = 0$|0.0
|Flash|Flash|Pressure Balance|  $\frac{P}{10000} - \frac{(P)}{10000} = 0$|0.0
|Flash|Flash|Temperature Balance|  $\frac{T}{1000} - \frac{T}{1000} = 0$|0.0
|Flash|Flash|Heat Balance|  $h + \frac{Q}{n} - (h * VF + h * 1 - VF) = 0$|-5.4569682106375694e-12
|Flash|Flash|Mass Balance|  $VF - VF = 0$|0.0
|Flash|Flash|Pressure Balance|  $\frac{P}{10000} - \frac{(P)}{10000} = 0$|0.0
|Flash|Flash|Temperature Balance|  $\frac{T}{1000} - \frac{T}{1000} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{Benzene} - nL_{Benzene} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{Benzene} - n_{Benzene} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{Toluene} - nL_{Toluene} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{Toluene} - n_{Toluene} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{P-xylene} - nL_{P-xylene} = 0$|0.0
|Flash|Flash|Equilibrium|  $n_{P-xylene} - n_{P-xylene} = 0$|0.0
